In [7]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']


from mrjob.job import MRJob
import re

import os
os.getcwd() 


'/home/vagrant/miniprojects/mr'

In [10]:
from mrjob.job import MRJob

In [2]:
print 'a'

a


In [3]:
%%file MRDoubleLink.py

from mrjob.job import MRJob
from mrjob.step import MRStep
from heapq import nsmallest
import re, mwparserfromhell, math
import xml.etree.ElementTree as ET

HEAD = re.compile('.*<page>.*')
TAIL = re.compile('.*</page>.*')
BAD1 = re.compile('(User(\stalk)?:.*)|(Wikipedia(\stalk)?:.*)|(File(\stalk)?:.*)|(MediaWiki(\stalk)?:.*)|(Template(\stalk)?:.*)|(Help(\stalk)?:.*)|(Category(\stalk)?:.*)|(Portal(\stalk)?:.*)|(Book(\stalk)?:.*)|(Draft(\stalk)?:.*)|(Education Program(\stalk)?:.*)|(TimedText(\stalk)?:.*)|(Module(\stalk)?:.*)|(Topic(\stalk)?:.*)|(Talk(\stalk)?:.*)')

BAD2 = re.compile('.*:.*')
BAD3 = re.compile('(Wikipedia:.*)|(WP:.*)|(Project:.*)|(File:.*)|(Image:.*)|(MediaWiki:.*)|(Template:.*)|(Help:.*)|(Category:.*)|(Portal:.*)|(Book:.*)|(Draft:.*)|(Education Program:.*)|(TimedText:.*)|(Module:.*)|(Topic:.*)|(WT:.*)|(Talk:.*)')
BAD4 = re.compile('(Category:.*)|(Talk:.*)|(Help:.*)|(Files:.*)|(Template:.*)')

BAD = BAD1

class MRDoubleLink(MRJob):
    
    def steps(self):
        return [#MRStep(mapper       = self.test,
                #       mapper_final = self.test_final),
                MRStep(mapper_init  = self.emit_init,
                       mapper       = self.emit,
                       reducer      = self.multiply),
                MRStep(reducer      = self.add),
                MRStep(mapper       = self.mini_heap,
                       reducer      = self.grand_heap)]
    
    def test(self, _, __):
        pass
    
    def test_final(self):
        yield ((0,0), 1)
        yield ((1,0), 2)
        yield ((0,1), 3)
        yield ((1,1), 4)
        
    def emit_init(self):
        self.temp = ''
        
    def emit(self, _, line):
        self.temp += line
        if TAIL.match(line):
            page = self.temp
            self.temp = ''
            # if HEAD.match(page):
                # x = ET.fromstring(page).find('revision').findtext('text')
                # x = mwparserfromhell.parse(x).filter_text()
                # x = " ".join(" ".join(i.value.split()) for i in x)   
            if HEAD.match(page):
                try:
                    tree = ET.fromstring(page)
                    home = str(tree.findtext('title').encode('utf-8'))
                    if not BAD.match(home):
                        text = tree.find('revision').findtext('text')
                        x = mwparserfromhell.parse(text).filter_wikilinks()
                        x = [str(i.title.encode('utf-8')) for i in x]
                        x = [i.lower() for i in x if not BAD.match(i)]
                        x = set(x)

                        weight = 1./(len(x)+10.)

                        for dest in x:
                            yield dest, ('A', (home.lower(), weight))
                            yield home.lower(), ('B', (dest, weight))

                except:
                    pass
        
    def multiply(self, hook, values):
        A_col, B_row = [], []
        for x in values:
            if x[0] == 'A':
                A_col.append(x[1])
            else:
                B_row.append(x[1])
        for (i, a) in A_col:
            for (k, b) in B_row:
                # if i <= k:
                    # first = i
                    # second = k
                # else:
                #     first = k
                #     second = i
                yield ((i, k), a*b)
                
    def add(self, i_k, x):
        yield (None, (-sum(x), i_k))
        
    def mini_heap(self, _, x):
        yield (None, nsmallest(100, x))
        
    def grand_heap(self, _, x):
        b = []
        for val, key in nsmallest(100, x):
            b.append((key,-val))
        
        yield None, b

        
if __name__ == '__main__':
    MRDoubleLink.run()

###########################################
###########################################


Overwriting MRDoubleLink.py


In [5]:
! python MRDoubleLink.py -r local /home/vagrant/miniprojects/mr/data_simple --output-dir 'DoubleLink2'

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/MRDoubleLink.vagrant.20160522.020215.587826
writing wrapper script to /tmp/MRDoubleLink.vagrant.20160522.020215.587826/setup-wrapper.sh

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to /tmp/MRDoubleLink.vagrant.20160522.020215.587826/step-0-mapper_part-00000
> sh -ex setup-wrapper.sh /opt/conda/bin/python MRDoubleLink.py --step-num=0 --mapper /tmp/MRDoubleLink.vagrant.20160522.020215.587826/input_part-00000 > /tmp/MRDoubleLink.vagrant.20160522.020215.587826/step-0-mapper_part-00000
writing to /tmp/MRDoubleLink.vagrant.20160522.020215.587826/step-0-mapper_part-00001
> sh -ex setup-wrapper.sh /opt/conda/bin/python MRDoubleLink.py --step-num=0 --mapper /tmp/MRDoubleLink.vagrant.20160522.020215.587826/input_part-000

In [6]:
list = [[["billy graham center", "billy graham center"], 0.0746524964439403], [["dolj county", "dolj county"], 0.06939402003707743], [["communes of the pyr\u00e9n\u00e9es-atlantiques department", "france"], 0.06837463861122495], [["communes of the yonne department", "france"], 0.06837439925893368], [["list of cities in illinois", "illinois"], 0.06834971545371495], [["communes of the pyr\u00e9n\u00e9es-atlantiques department", "departments of france"], 0.06829071150602123], [["communes of the yonne department", "departments of france"], 0.0682741134806818], [["communes of the pyr\u00e9n\u00e9es-atlantiques department", "pyr\u00e9n\u00e9es-atlantiques"], 0.06823432166799646], [["communes of the pyr\u00e9n\u00e9es-atlantiques department", "commune in france"], 0.06818766246680198], [["communes of the yonne department", "yonne"], 0.06816039480764109], [["communes of the yonne department", "communes of france"], 0.06815641103247087], [["list of cities in illinois", "united states"], 0.06767464794901615], [["communes of the vend\u00e9e department", "france"], 0.06760761202675272], [["communes of the vend\u00e9e department", "pays de la loire"], 0.06744877012294435], [["communes of the vend\u00e9e department", "communes of france"], 0.06727215408620821], [["communes of the vend\u00e9e department", "vend\u00e9e"], 0.06726354971518773], [["communes of the allier department", "france"], 0.06723459812712236], [["communes of the allier department", "allier"], 0.06707140353565576], [["communes of the allier department", "departments of france"], 0.0670027843001758], [["communes of the allier department", "communes of france"], 0.06681841851846487], [["list of cities in illinois", "city"], 0.06589940852599267], [["list of cities in indiana", "indiana"], 0.06580154439249318], [["list of cities in indiana", "united states"], 0.06508685167780047], [["communes of the mayenne department", "france"], 0.06467249833618108], [["communes of the mayenne department", "pays de la loire"], 0.06451709818078091], [["communes of the mayenne department", "communes of france"], 0.06431580993601439], [["communes of the mayenne department", "mayenne"], 0.06430651970933286], [["dolj county", "romania"], 0.062095998909269055], [["communes of the aisne department", "france"], 0.061199153273699736], [["communes of the aisne department", "departments of france"], 0.061142260380268376], [["communes of the aisne department", "regions of france"], 0.06106175061225861], [["communes of the aisne department", "aisne"], 0.061058668758215216], [["communes of the aisne department", "communes of france"], 0.06105640873191673], [["communes of the aisne department", "picardie"], 0.061053326877873335], [["list of cities in iowa", "iowa"], 0.060784993720297116], [["communes of the pas-de-calais department", "france"], 0.06073932355834233], [["communes of the pas-de-calais department", "departments of france"], 0.06063735714868362], [["communes of the pas-de-calais department", "nord-pas-de-calais"], 0.06063735714868362], [["communes of the pas-de-calais department", "pas-de-calais"], 0.060628323447342224], [["communes of the pas-de-calais department", "communes of france"], 0.060576307087633556], [["communes of the ard\u00e8che department", "ard\u00e8che"], 0.0605168736560756], [["communes of the pas-de-calais department", "regions of france"], 0.060516549851405727], [["list of cities in iowa", "united states"], 0.06050456131882248], [["list of cities in iowa", "city"], 0.060418670918652724], [["communes of the gironde department", "france"], 0.06040630747881124], [["communes of the ard\u00e8che department", "france"], 0.060296494607139005], [["communes of the gironde department", "aquitaine"], 0.06027690696479533], [["communes of the gironde department", "departments of france"], 0.06027690696479533], [["communes of the gironde department", "regions of france"], 0.060185473850106405], [["communes of the gironde department", "gironde"], 0.06018088690455345], [["communes of the gironde department", "communes of france"], 0.06017752314448128], [["communes of the calvados department", "france"], 0.059947927517875345], [["communes of the calvados department", "calvados (department)"], 0.05988505816040018], [["communes of the calvados department", "basse-normandie"], 0.05987891093433594], [["communes of the calvados department", "regions of france"], 0.059837377753107046], [["communes of the calvados department", "communes of france"], 0.05983123052704281], [["communes of the calvados department", "departments of france"], 0.05983123052704281], [["communes of the sarthe department", "france"], 0.059092753229189374], [["list of cities in oklahoma", "united states"], 0.059070515405440775], [["list of cities in oklahoma", "oklahoma"], 0.05904957738199019], [["communes of the sarthe department", "departments of france"], 0.058972000965579974], [["communes of the sarthe department", "pays de la loire"], 0.058972000965579974], [["communes of the sarthe department", "regions of france"], 0.05884161547805163], [["communes of the sarthe department", "sarthe"], 0.05883507440008198], [["communes of the sarthe department", "communes of france"], 0.05883027760957091], [["communes of the ard\u00e8che department", "departments of france"], 0.05830202680078439], [["communes of the ain department", "france"], 0.05819088269191497], [["communes of the vaucluse department", "france"], 0.05816218147760935], [["communes of the ain department", "regions of france"], 0.05809228397262659], [["communes of the ain department", "departments of france"], 0.0580820668589612], [["communes of the ain department", "rh\u00f4ne-alpes"], 0.0580820668589612], [["communes of the ain department", "ain"], 0.05804870340897819], [["communes of the ain department", "communes of france"], 0.05804438078396591], [["dolj county", "commune in romania"], 0.05802775655716839], [["list of cities in oklahoma", "city"], 0.057885302994497005], [["communes of the vaucluse department", "provence-alpes-c\u00f4te d'azur"], 0.057579337840873504], [["communes of the vaucluse department", "vaucluse"], 0.05751574707368992], [["kiss (band)", "kiss (band)"], 0.057483389574816295], [["communes of the vaucluse department", "communes of france"], 0.05720508634386752], [["list of cities in idaho", "idaho"], 0.055922694796753325], [["communes of the alpes-maritimes department", "france"], 0.055805588816024605], [["list of cities in idaho", "united states"], 0.05579352323901034], [["communes of the alpes-maritimes department", "alpes-maritimes"], 0.05570006322631002], [["list of cities in arkansas", "arkansas"], 0.05566332274775213], [["list of cities in arkansas", "united states"], 0.055473605426560706], [["communes of the alpes-maritimes department", "communes of france"], 0.05541022891412934], [["communes of the alpes-maritimes department", "provence-alpes-c\u00f4te d'azur"], 0.055218002086472706], [["communes of the alpes-maritimes department", "departments of france"], 0.054912656484544776], [["list of cities in idaho", "city"], 0.05482154369560222], [["list of cities in arkansas", "city"], 0.05475076940874899], [["communes of the alpes-maritimes department", "regions of france"], 0.054565046304532364], [["list of japan international footballers", "association football"], 0.05380394909471659], [["list of civil parishes in essex", "essex"], 0.05372816983569709], [["list of japan international footballers", "japan national football team"], 0.053626044807223265], [["list of japan international footballers", "japan"], 0.05341062426720476], [["major league baseball", "major league baseball"], 0.053256785848419703], [["list of cities in kentucky", "kentucky"], 0.0532342993534176], [["list of cities in kentucky", "united states"], 0.05291187337229633], [["list of cities in kentucky", "city"], 0.052876036047973175], [["arrondissement of largenti\u00e8re", "france"], 0.0523237134317062]]

In [7]:
len(list)

100

In [8]:
list[:5]

[[['billy graham center', 'billy graham center'], 0.0746524964439403],
 [['dolj county', 'dolj county'], 0.06939402003707743],
 [['communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'],
  0.06837463861122495],
 [['communes of the yonne department', 'france'], 0.06837439925893368],
 [['list of cities in illinois', 'illinois'], 0.06834971545371495]]

In [9]:
list2 = [((a,b),c) for [a,b],c in list]

In [10]:
print list2

[(('billy graham center', 'billy graham center'), 0.0746524964439403), (('dolj county', 'dolj county'), 0.06939402003707743), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'france'), 0.06837463861122495), (('communes of the yonne department', 'france'), 0.06837439925893368), (('list of cities in illinois', 'illinois'), 0.06834971545371495), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'departments of france'), 0.06829071150602123), (('communes of the yonne department', 'departments of france'), 0.0682741134806818), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'pyr\\u00e9n\\u00e9es-atlantiques'), 0.06823432166799646), (('communes of the pyr\\u00e9n\\u00e9es-atlantiques department', 'commune in france'), 0.06818766246680198), (('communes of the yonne department', 'yonne'), 0.06816039480764109), (('communes of the yonne department', 'communes of france'), 0.06815641103247087), (('list of cities in illinois', 'united states'), 0.06

In [3]:
%%file MRStatsSimple.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import heapq
from lxml import etree
from StringIO import StringIO
import sys 
import numpy as np
import xml.etree.ElementTree as ET
import mwparserfromhell
    
WORD_RE = re.compile(r"[\w]+")
        
class MRStatsSimple(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_comb_init,
                   mapper=self.mapper_comb_lines),
            MRStep(mapper=self.mapper_get_words,
                   reducer=self.reducer_count_words)
        ]

    def mapper_comb_init(self):
        self.in_page = False
        self.page = []
        
    def mapper_comb_lines(self, _, line):
        #line = line.strip()
        if '<page>' in line:
            self.in_page = True
        if '</page>' in line and self.in_page:
            self.page.append(line)
            self.in_page=False
            yield None, self.page
            self.page = []
        if self.in_page:
            self.page.append(line)
            
    def mapper_get_words(self, _, page):

        tree = ET.fromstring(''.join(page).encode('utf-8').strip())
        root = tree
        for x in root.iter(tag = 'text'):
            if not x.text:
                x.text = ''
            wikicode = mwparserfromhell.parse(x.text)
            #wikicode = x.text.encode('utf-8')
            wikilinks = wikicode.filter_wikilinks()
            wikilinks2 = list(set(i.encode('utf-8') for i in wikilinks))

            yield None, len(wikilinks2)

###simple
#        tree = ET.fromstring(''.join(page).encode('utf-8').strip())
#        root = tree
#        for x in root.iter(tag = 'text'):
#            if not x.text:
#                x.text = ''
#            wiki = x.text.encode('ascii','ignore')

### nometa

#        tree = ET.fromstring(''.join(page).encode('utf-8').strip())
#        root = tree
#        for x in root.iter(tag = 'text'):
#            if not x.text:
#                x.text = ''
#            wikicode = mwparserfromhell.parse(x.text)
#            wikicode2 = wikicode.filter_text()
#            wikicode3 = [s.encode('utf-8') for s in wikicode2]
            
    def reducer_count_words(self, _, counts):
        list = []
        for i in counts:
            list.append(int(i))
        
        yield None, (sum(list), np.mean(list), np.std(list)
                     , np.percentile(list, 5), np.percentile(list, 25), np.percentile(list, 50)
                     , np.percentile(list, 75), np.percentile(list, 95))

if __name__ == '__main__':
    MRStatsSimple.run()

Overwriting MRStatsSimple.py


In [4]:
! python MRStatsSimple.py -r local /home/vagrant/miniprojects/mr/data_english --output-dir 'english_stat_new'

using configs in /home/vagrant/.mrjob.conf
creating tmp directory /tmp/MRStatsSimple.vagrant.20160522.141424.059859
writing wrapper script to /tmp/MRStatsSimple.vagrant.20160522.141424.059859/setup-wrapper.sh

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

^CTraceback (most recent call last):
  File "MRStatsSimple.py", line 84, in <module>
    MRStatsSimple.run()
  File "/opt/conda/lib/python2.7/site-packages/mrjob/job.py", line 461, in run
    mr_job.execute()
  File "/opt/conda/lib/python2.7/site-packages/mrjob/job.py", line 479, in execute
    super(MRJob, self).execute()
  File "/opt/conda/lib/python2.7/site-packages/mrjob/launch.py", line 153, in execute
    self.run_job()
  File "/opt/conda/lib/python2.7/site-packages/mrjob/launch.py", line 216, in run_job
    runner.run()
  File "/opt

In [14]:
print 'a'

a


In [15]:
%%file MRStats.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, heapq, Queue
from xml.etree import ElementTree
import mwparserfromhell, random, math

WORD_RE = re.compile(r"\w+")
START_RE = re.compile('.*<page>.*')
END_RE = re.compile('.*</page>.*')
text_start_re = re.compile('.*<text.*')
text_end_re = re.compile('.*</text>.*')

class MRStats(MRJob):

    def steps(self):
        return [
            MRStep(mapper_init = self.mapper_xml_init,
                   mapper = self.mapper_xml,
                   mapper_final = self.mapper_xml_final,
                   reducer=self.reducer_compute_stats)
            ]

    def mapper_xml_init(self):
        self.queue = ''#Queue.Queue()
        self.sum_x = 0
        self.sum_x_sq = 0
        self.reservoir = []
        self.k = 1000
        # self.c = 0
        self.pagecount = 0
                
        
    def mapper_xml(self, _,line):
        self.queue += line#.put(line)
        if END_RE.match(line):
            # empty the queue
            # page = ''
            # while not self.queue.empty():
            #     page += self.queue.get()
            page = self.queue
            self.queue = ''
            if START_RE.match(page):
                self.pagecount += 1
            
                tree = ElementTree.fromstring(page)#.encode('utf-8'))
                text = tree.find('revision').findtext('text')
                links = mwparserfromhell.parse(text).filter_wikilinks()
                # try:
                links = [str(i.title.encode('utf-8')) for i in links]
                # except:
                    # links = []
                # if len(links) > 0:
                x = len(set(links))
                # else:
                #     x = 0

                r = random.random()
                self.sum_x += x
                self.sum_x_sq += x**2
                if len(self.reservoir) < self.k:
                    heapq.heappush(self.reservoir,(r,x))
                elif r > self.reservoir[0][0]:
                    heapq.heapreplace(self.reservoir,(r,x))
                        
    def mapper_xml_final(self):
        yield('mean var',(self.sum_x, self.sum_x_sq, self.pagecount))
        for r,x in self.reservoir:
            yield ('reservoir',(r,x))
      
    def reducer_compute_stats(self, key, vals):
        # optimization: sum the words we've seen so far
        if key == 'mean var':
            sum_x = 0
            sum_x_sq = 0
            pagecount = 0
            for val in vals:
                x, x_sq, count= val
                sum_x += x
                sum_x_sq += x_sq
                pagecount += count
            pagecount = float(pagecount)
            mean = sum_x/pagecount
            std_dev = math.sqrt(sum_x_sq/pagecount-mean**2)
            yield key,(('page count',int(pagecount)), ('mean',mean),('std dev',std_dev) )
        elif key == 'reservoir':
            counts = [x for r,x in vals]
            counts.sort()
            yield key, (('5th percentile',counts[int(round(len(counts)*0.05)-1)]),
            ('25th percentile',counts[int(round(len(counts)*0.25)-1)]),
            ('50th percentile',counts[int(round(len(counts)*0.5)-1)]),
            ('75th percentile',counts[int(round(len(counts)*0.75)-1)]),
            ('95th percentile',counts[int(round(len(counts)*0.95)-1)]) )

if __name__ == '__main__':
    MRStats.run()

Overwriting MRStats.py


In [16]:
! python stat.py -r local /home/vagrant/miniprojects/mr/data_english --output-dir 'english_stat_new2'

Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/site.py", line 62, in <module>
    import os
  File "/opt/conda/lib/python2.7/os.py", line 49, in <module>
    import posixpath as path
  File "/opt/conda/lib/python2.7/posixpath.py", line 15, in <module>
    import stat
  File "stat.py", line 3, in <module>
    from mrjob.step import MRStep
ImportError: No module named mrjob.step


In [17]:
! python MRDoubleLink.py -r local /home/vagrant/miniprojects/mr/data_simple --output-dir 'DoubleLink2'

Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/site.py", line 62, in <module>
    import os
  File "/opt/conda/lib/python2.7/os.py", line 49, in <module>
    import posixpath as path
  File "/opt/conda/lib/python2.7/posixpath.py", line 15, in <module>
    import stat
  File "stat.py", line 3, in <module>
    from mrjob.step import MRStep
ImportError: No module named mrjob.step
